In [ ]:
import os
import re
import json
import hashlib
import webbrowser
import random
from datetime import datetime

import pandas as pd


# =========================
# CONFIG
# =========================
SHEET_ID = "1oEHfYvcQiNMNgCW-N_RLZ1bRY7UjBSLrPDT8yCED5ZE"
API_URL = "https://script.google.com/a/macros/resultate.com.br/s/AKfycbzTURe7wE_DQ4XPRiGeM44yA06srf87Ovrbxrh8ReHe9zxBaY-VjbzxEukyot821AOg/exec"
API_TOKEN = "Resultate@2026_" 

STATUS_VALIDOS = {"ok", "pular", "revisar", "urgente"}

IGNORAR_ITENS = {
    "mensagem da semana - alinhamento",
    "mensagem da semana",
    "alinhamento da semana",
    "alinhamento",
    "mensagem simples",
}

REGRAS_REESCRITA = [
    ("cronograma abril: fazer",
     "Estamos montando o cronograma de abril. Tem alguma prioridade/tema importante para incluir?"),
]


# =========================
# HELPERS (texto / ids)
# =========================
def normalizar(txt: str) -> str:
    return " ".join(str(txt).strip().lower().split())

def _slug(txt: str) -> str:
    return " ".join(str(txt).strip().lower().split())

def _hash8(txt: str) -> str:
    return hashlib.md5(txt.encode("utf-8")).hexdigest()[:8]

def escape_html(txt: str) -> str:
    return str(txt).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

def escape_attr_multiline(txt: str) -> str:
    return str(txt).replace("&", "&amp;").replace('"', "&quot;").replace("<", "&lt;").replace(">", "&gt;").replace("\n", "&#10;")

def pegar_status(valor) -> str:
    if pd.isna(valor): return "ok"
    s = normalizar(valor)
    if not s or s not in STATUS_VALIDOS: return "ok"
    return s

def deve_ignorar(linha: str) -> bool:
    t = normalizar(linha)
    if t.startswith("-"): t = t[1:].strip()
    if not t or t == "nan": return True
    for item in IGNORAR_ITENS:
        if normalizar(item) in t: return True
    return False

def filtrar_pendencias(linhas):
    filtradas = []
    for linha in linhas:
        if not deve_ignorar(linha): filtradas.append(linha.strip())
    return filtradas

def reescrever_item(linha: str) -> str:
    t = normalizar(linha)
    if t.startswith("-"): t = t[1:].strip()
    for alvo, novo in REGRAS_REESCRITA:
        if normalizar(alvo) in t: return novo
    return linha

def extrair_itens(pendencias) -> list[str]:
    raw = "" if pendencias is None or pd.isna(pendencias) else str(pendencias)
    linhas = [l.strip() for l in raw.replace("\r", "\n").split("\n") if l.strip()]
    out = []
    for l in linhas:
        if l.startswith("-"): l = l[1:].strip()
        if l: out.append(l)
    out = filtrar_pendencias(out)
    return [reescrever_item(x) for x in out]

def precisa_mensagem_simples(pendencias) -> bool:
    if pd.isna(pendencias) or not str(pendencias).strip(): return True
    texto = normalizar(pendencias)
    if texto in {normalizar(x) for x in IGNORAR_ITENS}: return True
    return False

def criar_checklist(pendencias_raw) -> str:
    itens = extrair_itens(pendencias_raw)
    if not itens: return ""
    linhas = []
    for it in itens:
        linha = it.strip()
        if not linha or normalizar(linha) == "nan": continue
        if linha.endswith("."): linha = linha[:-1].strip()
        linhas.append(f"☑️ {linha}")
    return "\n".join(linhas).strip()

def obter_saudacao() -> str:
    hora = datetime.now().hour
    if hora < 12: return "Bom dia"
    elif hora < 18: return "Boa tarde"
    else: return "Boa noite"

def obter_mensagem_motivacional() -> str:
    mensagens = [
        "🚀 'O único lugar onde o sucesso vem antes do trabalho é no dicionário.'",
        "🌟 'A persistência é o caminho do êxito. Bora pra cima!'",
        "💪 'Bora fazer acontecer! Cada mensagem enviada é um passo rumo ao resultado.'",
        "🔥 'Foco, força e café! Uma excelente semana de trabalho!'",
        "✨ 'Pequenos progressos diários levam a grandes resultados.'",
        "🏆 'Organização é a chave. Vamos dominar essa semana!'"
    ]
    return random.choice(mensagens)

def criar_mensagem(nome, pendencias):
    nome_time = f"TIME {nome}".upper()
    checklist = criar_checklist(pendencias)
    saudacao = obter_saudacao()

    if not checklist.strip():
        return f"""Olá, {nome_time}! {saudacao}! 😊

Como vão?

Passando para desejar uma ótima semana e verificar se precisam de algo.

Estamos à disposição.

Vamos juntos! 🚀"""

    return f"""Olá, {nome_time}! {saudacao}! 😊

Como vão?

Espero que tenham uma ótima semana!

Passando para deixar nossos alinhamentos da semana:

{checklist}

Qualquer dúvida ou informação, só me chamar.

Vamos juntos! 🚀"""

def gerar_run_hash(clientes) -> str:
    partes = []
    for _, c in clientes.iterrows():
        partes.append({
            "nome": str(c.get("nome", "")),
            "grupo": str(c.get("grupo_whatsapp", "")),
            "pendencias": str(c.get("pendencias", "")),
            "status": str(c.get("status", "ok")),
            "cliente_id": str(c.get("cliente_id", "")),
        })
    payload = json.dumps(partes, ensure_ascii=False, sort_keys=True)
    payload += "@" + datetime.now().strftime("%Y%m%d%H%M%S")
    return hashlib.md5(payload.encode("utf-8")).hexdigest()

# =========================
# HTML
# =========================
def gerar_html(clientes: pd.DataFrame) -> str:
    clientes = clientes.copy()
    if "status" not in clientes.columns: clientes["status"] = "ok"
    clientes["status_norm"] = clientes["status"].apply(pegar_status)
    ordem = {"urgente": 0, "revisar": 1, "ok": 2, "pular": 9}
    clientes["prioridade"] = clientes["status_norm"].map(ordem).fillna(2).astype(int)

    if "nome" in clientes.columns:
        clientes = clientes.sort_values(by=["prioridade", "nome"], ascending=[True, True])
    else:
        clientes = clientes.sort_values(by=["prioridade"], ascending=[True])

    RUN_HASH = gerar_run_hash(clientes)
    motivacional = obter_mensagem_motivacional()

    html = f"""<!DOCTYPE html>
<html lang="pt-BR">
<head>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;800&display=swap" rel="stylesheet">
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Mensagens de Segunda-feira</title>

  <style>
    :root {{
      /* 💡 MUDANÇA DE COR: Cinza Bem Suave */
      --bg: #F4F6F8; 
      
      --card: #FFFFFF;
      --text: #111111;
      --muted: #6B6B6B;
      --yellow: #FFD200;
      --yellow-2: #FFC400;
      --border: #EAEAEA;
      --soft: #F7F7F7;
      --ok: #2E7D32;
      --urg: #C62828;
      --wpp: #25D366;
    }}

    * {{ margin: 0; padding: 0; box-sizing: border-box; }}

    body {{ font-family: 'Inter', sans-serif; background: var(--bg); padding: 40px; min-height: 100vh; color: var(--text); }}
    .container {{ max-width: 1100px; margin: 0 auto; }}

    .header {{ background: var(--card); padding: 32px; border-radius: 16px; margin-bottom: 32px; border: 1px solid var(--border); position: sticky; top: 20px; z-index: 100; box-shadow: 0 10px 30px rgba(0,0,0,0.1); }}
    .header h1 {{ font-size: 2.2rem; font-weight: 800; color: #000; margin-bottom: 8px; }}
    
    .motivational-text {{ font-size: 1.1rem; color: #D84315; font-weight: 700; font-style: italic; margin-bottom: 15px; background: #FFF3E0; padding: 10px 15px; border-radius: 8px; border-left: 4px solid #FF9800; }}
    
    .header p.meta-info {{ color: var(--muted); font-size: 0.95rem; margin-bottom: 15px; }}
    
    .progress-wrap {{ background: var(--soft); border-radius: 10px; padding: 15px; }}
    .progress-text {{ font-weight: 800; color: var(--ok); font-size: 1rem; margin-bottom: 8px; display: flex; justify-content: space-between; }}
    .progress-track {{ background: #ddd; height: 12px; border-radius: 10px; overflow: hidden; }}
    .progress-bar {{ background: var(--ok); height: 100%; width: 0%; transition: width 0.4s ease; }}

    .instructions {{ background: var(--card); border-left: 6px solid var(--yellow); padding: 24px; border-radius: 14px; margin-bottom: 32px; border: 1px solid var(--border); box-shadow: 0 5px 15px rgba(0,0,0,0.05); }}
    .instructions h3 {{ font-weight: 800; margin-bottom: 12px; }}
    .instructions ol {{ margin-left: 20px; line-height: 1.8; color: var(--muted); }}

    .message-card {{ background: var(--card); padding: 32px; border-radius: 18px; margin-bottom: 28px; border: 1px solid var(--border); transition: 0.2s ease; box-shadow: 0 5px 15px rgba(0,0,0,0.05); }}
    .message-card:hover {{ transform: translateY(-4px); box-shadow: 0 12px 30px rgba(0,0,0,0.12); }}
    
    .client-name {{ font-size: 1.6rem; font-weight: 800; margin-bottom: 6px; }}
    .group-name {{ color: var(--muted); font-size: 0.95rem; margin: 8px 0 14px 0; font-style: italic; }}
    
    .pill {{ padding: 6px 14px; border-radius: 999px; font-size: 0.75rem; font-weight: 800; display: inline-block; margin-right: 8px; }}
    .message-type {{ background: var(--ok); color: white; font-weight: 700; }}
    .message-type-simple {{ background: var(--yellow); color: #000; font-weight: 900; }}
    .status-ok {{ background: var(--ok); color: white; }}
    .status-revisar {{ background: var(--yellow); color: #000; }}
    .status-urgente {{ background: var(--urg); color: white; }}

    .message-preview {{ margin-top: 22px; }}
    .message-content {{ background: var(--soft); padding: 22px; border-radius: 12px; line-height: 1.8; white-space: pre-line; word-break: break-word; margin-bottom: 18px; font-size: 0.95rem; border: 1px solid var(--border); }}

    .copy-button {{ background: var(--yellow); color: #000; border: none; padding: 14px; border-radius: 10px; font-weight: 900; cursor: pointer; width: 100%; transition: 0.2s ease; margin-bottom: 10px; display: flex; justify-content: center; align-items: center; gap: 8px; }}
    .copy-button:hover {{ background: var(--yellow-2); transform: scale(1.02); }}
    
    .btn-wpp {{ background: var(--wpp); color: white; }}
    .btn-wpp:hover {{ background: #128C7E; }}
    
    .btn-dark {{ background: #111; color: #fff; }}
    .btn-dark:hover {{ background: #000; }}
    .copied {{ background: var(--ok) !important; color: white !important; }}

    .sent-info {{ margin-top: 10px; color: var(--muted); font-size: 0.9rem; display: none; }}

    .todo {{ margin-top: 16px; padding: 16px; border-radius: 12px; border: 1px solid var(--border); background: #fff; }}
    .todo h4 {{ font-size: 0.95rem; font-weight: 900; margin-bottom: 10px; color: #000; }}
    .todo-item {{ display: flex; align-items: flex-start; gap: 10px; padding: 10px 8px; border-radius: 10px; }}
    .todo-item:hover {{ background: #FAFAFA; }}
    .todo-item input {{ width: 18px; height: 18px; margin-top: 3px; accent-color: var(--yellow); }}
    .todo-text {{ color: #111; font-size: 0.95rem; line-height: 1.4; word-break: break-word; flex: 1; }}
    .todo-text.done {{ text-decoration: line-through; color: var(--muted); }}
    .todo-actions {{ display: flex; gap: 10px; margin-top: 10px; }}
    .mini {{ padding: 10px 12px; border-radius: 10px; border: 1px solid var(--border); background: #fff; cursor: pointer; font-weight: 800; }}
    .mini:hover {{ background: #FAFAFA; }}
  </style>
</head>

<body>
  <div class="container">
    <div class="header">
      <h1>🗂️ Mensagens de Segunda-feira</h1>
      
      <div class="motivational-text">{escape_html(motivacional)}</div>
      
      <p class="meta-info">Gerado em {datetime.now().strftime("%d/%m/%Y às %H:%M")} • RUN: {RUN_HASH[:8]}</p>
      
      <div class="progress-wrap">
        <div class="progress-text">
          <span>Progresso de Envios</span>
          <span><span id="prog-count">0</span> / <span id="prog-total">0</span></span>
        </div>
        <div class="progress-track">
          <div class="progress-bar" id="prog-bar"></div>
        </div>
      </div>
    </div>

    <div class="instructions">
      <h3>💡 Como usar:</h3>
      <ol>
        <li>Clique em <strong>"📱 ENVIAR NO WHATSAPP"</strong> (Ele já abre o App com o texto copiado!)</li>
        <li>Escolha o grupo do cliente e envie</li>
        <li>Volte aqui e clique em <strong>"MARCAR COMO ENVIADO"</strong></li>
      </ol>
    </div>
"""

    # CARDS
    for _, cliente in clientes.iterrows():
        status = pegar_status(cliente.get("status", "ok"))
        if status == "pular": continue

        status_label = {"ok": "✅ OK", "revisar": "🟡 REVISAR", "urgente": "🔴 URGENTE"}.get(status, "✅ OK")
        status_class = {"ok": "status-ok", "revisar": "status-revisar", "urgente": "status-urgente"}.get(status, "status-ok")

        nome = str(cliente.get("nome", "")).strip()
        grupo = str(cliente.get("grupo_whatsapp", "Grupo não especificado")).strip()
        pendencias = cliente.get("pendencias", "")

        mensagem = criar_mensagem(nome, pendencias)
        is_simple = precisa_mensagem_simples(pendencias)
        tipo_label = "Mensagem Simples" if is_simple else "Mensagem com Pendências"

        cliente_id = str(cliente.get("cliente_id", "")).strip()
        client_key = cliente_id if cliente_id else f"{nome}__{grupo}".strip().lower().replace(" ", "_")
        msg_attr = escape_attr_multiline(mensagem)

        itens = extrair_itens(pendencias)
        checklist_html = ""
        if itens:
            rows = []
            for it in itens:
                it_id = _hash8(_slug(it))
                rows.append(
                    f'<label class="todo-item">'
                    f'  <input type="checkbox" data-item="{it_id}" onchange="toggleItem(this)">'
                    f'  <div class="todo-text" data-text="{it_id}">{escape_html(it)}</div>'
                    f'</label>'
                )
            checklist_html = f"""
      <div class="todo" data-client="{escape_attr_multiline(client_key)}">
        <h4>Checklist (item por item)</h4>
        {''.join(rows)}
        <div class="todo-actions">
          <button class="mini" onclick="marcarTodos('{escape_attr_multiline(client_key)}')">Marcar todos</button>
          <button class="mini" onclick="limparTodos('{escape_attr_multiline(client_key)}')">Limpar</button>
        </div>
      </div>"""

        html += f"""
    <div class="message-card" data-key="{escape_attr_multiline(client_key)}" data-nome="{escape_attr_multiline(nome)}" data-grupo="{escape_attr_multiline(grupo)}" data-id="{escape_attr_multiline(cliente_id)}">
      <div class="client-name">{escape_html(nome)}</div>
      <span class="pill {status_class}">{status_label}</span>
      <div class="group-name">📱 {escape_html(grupo)}</div>
      <span class="pill {('message-type-simple' if is_simple else 'message-type')}">{tipo_label}</span>

      <div class="message-preview">
        <div class="message-content">{escape_html(mensagem)}</div>

        <button class="copy-button btn-wpp" data-msg="{msg_attr}" onclick="enviarWhatsapp(this)">
          📱 ENVIAR DIRETO NO WHATSAPP
        </button>

        <button class="copy-button" data-msg="{msg_attr}" onclick="copyMessage(this)">
          📋 APENAS COPIAR TEXTO
        </button>

        <button class="copy-button btn-dark btn-marcar" onclick="marcarEnviado(this)">
          ✅ MARCAR COMO ENVIADO
        </button>

        <button class="copy-button btn-dark btn-reenviar" style="display:none;" onclick="enviarNovamente(this)">
          🔄 ENVIAR NOVAMENTE
        </button>

        <div class="sent-info">
          ✅ Último envio: <span class="sent-date"></span>
          <div style="margin-top:6px;">🧾 Enviado nesta execução: <span class="sent-run"></span></div>
        </div>
        {checklist_html}
      </div>
    </div>"""

    # SCRIPTS
    html += f"""
  </div>
  <script>
    const RUN_HASH = "{RUN_HASH}";
    const API_URL = "{API_URL}";
    const API_TOKEN = "{API_TOKEN}";

    async function postToSheet(payload) {{
      try {{
        await fetch(API_URL, {{
          method: "POST", mode: "no-cors",
          headers: {{ "Content-Type": "text/plain;charset=utf-8" }}, 
          body: JSON.stringify({{ token: API_TOKEN, ...payload }})
        }});
        return true;
      }} catch (e) {{ console.error(e); return false; }}
    }}

    function storageKeyClient(key) {{ return "cliente_state_" + key; }}
    function storageKeyChecklist(key) {{ return "checklist_" + key; }}

    function getClientState(key) {{
      try {{ return JSON.parse(localStorage.getItem(storageKeyClient(key))) || {{ history: [] }}; }} 
      catch (e) {{ return {{ history: [] }}; }}
    }}
    function setClientState(key, state) {{ localStorage.setItem(storageKeyClient(key), JSON.stringify(state)); }}

    function enviarWhatsapp(button) {{
      const text = button.getAttribute("data-msg");
      const url = "https://api.whatsapp.com/send?text=" + encodeURIComponent(text);
      window.open(url, "_blank");
    }}

    function copyMessage(button) {{
      const text = button.getAttribute("data-msg");
      navigator.clipboard.writeText(text).then(() => {{
        const originalText = button.innerHTML;
        button.innerHTML = "✅ COPIADO!";
        button.classList.add("copied");
        setTimeout(() => {{ button.innerHTML = originalText; button.classList.remove("copied"); }}, 2000);
      }});
    }}

    function atualizarProgresso() {{
      const cards = document.querySelectorAll(".message-card");
      const total = cards.length;
      let concluidos = 0;

_IncompleteInputError: incomplete input (497727188.py, line 380)

In [ ]:
print("postToSheet")

postToSheet
